In [1]:
#coding=utf-8

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import os
import random
import time
import json
hd = {
    "User-Agent":
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:73.0) Gecko/20100101 Firefox/73.0"}


In [2]:
def gethtml(url):    # 获取网站 html 信息
    headers = {

        "Host": "www.ximalaya.com",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:73.0) Gecko/20100101 Firefox/73.0",

        }
    # 用的代理 ip，如果被封的，在http://www.xicidaili.com/换一个
    html = requests.get(url, headers=headers)   # 请求网页信息
    return html

In [3]:
def getNumOfEpisodes(book_Id):
    req = gethtml('https://www.ximalaya.com/'+'jiaoyu'+'/'+str(book_Id)+'/p'+str(1))

    result = req.text

    b = re.findall (u"专辑里的声音\(<!-- -->([\d]*)",result)

    return int(b[0])

In [12]:
def getid():    # 获取专辑的 id 和标题信息
    keyword = input('请输入你要查找的音频关键字:\n')     # 输入需要下载音频的关键字
    albumurl = 'https://www.ximalaya.com/search/album/{}/sc/p1'.format(keyword)    # 输入关键字，拼接链接
    html = gethtml(albumurl)
    f = open('file.txt','w',encoding='utf-8')
    f.write(html.text)
    f.close()

    soup = BeautifulSoup(html.text, 'lxml')
    info = soup.select('#searchPage div.search-type div.common-tab-content div.xm-loading ul div '
                       'a.xm-album-title.ellipsis-2')    # 提取音频文件的信息
    idinfo = re.compile('href="/.*?"').findall(str(info))     # 提取专辑中 id

    titleinfo = re.compile('title=".*?"').findall(str(info))     # 提取专辑中标题信息

    ids = []
    titles = []
    NumsOfEpisodes = []
    for j in idinfo:
        id = str(j)
        ids.append(id)
    for t in titleinfo:
        # 处理下标题，防止创建文件夹失败
        title = str(t).split('"')[1].replace('\\', ' ').replace('/', ' ').replace(':', ' ').replace('*', ' ')\
            .replace('?', ' ').replace('"', ' ').replace('<', ' ').replace('>', ' ').replace('|', ' ')
        titles.append(title)
    for k in ids:
        NumsOfEpisodes.append(getNumOfEpisodes(k))
    return titles, ids, NumsOfEpisodes

In [14]:
getid()

(['我当阴阳先生的那几年｜张二狗',
  '我当阴阳先生的那几年',
  '我当阴阳先生的那几年-张二狗',
  '我当阴阳先生的那几年【头陀渊】',
  '我当阴阳先生的那几年（屁屁球原声高清）',
  '我当阴阳先生的那几年-张二狗',
  '我当阴阳先生的那几年全集（屁屁球原声高清）',
  '我当阴阳先生的那几年',
  '我当阴阳先生的那几年 播讲 张二狗 作者 崔走召',
  '我当阴阳先生的那几年（麻小帅版）',
  '我当阴阳先生的那几年',
  '我当阴阳先生的那几年(多人小说)',
  '我当阴阳先生的那几年',
  '我当阴阳先生的那几年东北话版',
  '我当阴阳先生的那几年（屁屁球原声高清）',
  '我当阴阳先生的那几年 子光演播',
  '我当阴阳先生的那几年',
  '致敬经典《我当阴阳先生那几年》',
  '我当阴阳先生的那几年',
  '【女版】我当阴阳先生的那几年'],
 ['25480690',
  '7235715',
  '31099497',
  '3017390',
  '10651552',
  '32206242',
  '9283606',
  '35385797',
  '7421834',
  '3554258',
  '29148148',
  '11665044',
  '14593230',
  '21861246',
  '10661512',
  '30235254',
  '19516474',
  '34747919',
  '28795518',
  '31593518'],
 [194,
  205,
  194,
  205,
  289,
  194,
  289,
  250,
  48,
  259,
  150,
  204,
  105,
  289,
  90,
  98,
  291,
  24,
  6,
  291])

In [9]:
download_pool = []

In [6]:
def download(url):
    if url not in download_pool:
        req = gethtml(url)
        result = req.text
        result = json.loads(result)
        title = result['title']
        data_url = result['play_path_64']
        
        data = requests.get(data_url).content
        print(title)

        with open('./download/'+'%s.m4a'%title,'wb') as f:
            f.write(data)
        download_pool.append(url)
    else:
        print('Skip repeat downloading')

##复制list的五种方法：

1. a\[:\]

2. list(a)

3. a*1

4. copy.copy(a)

5. copy.deepcopy(a)

In [7]:
def get_EpisodeId(categray,bookID,page):

    req = gethtml('https://www.ximalaya.com/'+categray+'/'+str(bookID)+'/p'+str(page))

    result = req.text

    id_number = re.findall('/'+str(bookID)+'/([\d]*)',result)

    id_number = set(id_number)  

    id_number.remove('')
    numbers = [ int(x) for x in id_number ] 
    numbers = sorted(numbers)
    '''
    numbers_fixed = numbers[:]

    for i in range(len(numbers)):
        print(len(numbers))
        if numbers[i] in ID_pool:
            #print('Same')
            numbers_fixed.remove(numbers[i])
        else :
            #print('Add')
            ID_pool.append(numbers[i])

    print(len(numbers_fixed))
    '''
    id_number = [ str(x) for x in numbers ] 

    id = []
    for number in id_number:
        id.append('http://www.ximalaya.com/tracks/' + number + '.json')  # 这样构造url是不太严谨的，当然在这里是可以的，以后再探讨 url 的构造方法
    print(len(id))
    return id

In [ ]:

for url in get_EpisodeId('youshengshu',30586083,1):
    download(url)


for url in get_EpisodeId('youshengshu',30586083,1):
    download(url)